In [ ]:
!pip install kaggle
from google.colab import files

# Authenticate Kaggle API
!kaggle datasets download -d alfathterry/bbc-full-text-document-classification

# Unzip the downloaded dataset
!unzip bbc-full-text-document-classification.zip

# List the files in the unzipped directory
!ls bbc-full-text-document-classification


Dataset URL: https://www.kaggle.com/datasets/alfathterry/bbc-full-text-document-classification
License(s): MIT
  0% 0.00/1.84M [00:00<?, ?B/s]
100% 1.84M/1.84M [00:00<00:00, 27.4MB/s]
Archive:  bbc-full-text-document-classification.zip
  inflating: bbc_data.csv            
ls: cannot access 'bbc-full-text-document-classification': No such file or directory


In [ ]:
#Load the dataset
import pandas as pd
df = pd.read_csv(r'/content/bbc_data.csv')
df.head()

,data,labels
0,Musicians to tackle US red tape Musicians gro...,entertainment
1,"U2s desire to be number one U2, who have won ...",entertainment
2,Rocker Doherty in on-stage fight Rock singer ...,entertainment
3,Snicket tops US box office chart The film ada...,entertainment
4,"Oceans Twelve raids box office Oceans Twelve,...",entertainment


In [ ]:
df.isnull().sum()

data      0
labels    0
dtype: int64

In [ ]:
print(df.shape)
print(df.describe())
print(df.labels.value_counts())

(2225, 2)
                                                     data labels
count                                                2225   2225
unique                                               2126      5
top     Sir Paul rocks Super Bowl crowds  Sir Paul McC...  sport
freq                                                    2    511
labels
sport            511
business         510
politics         417
tech             401
entertainment    386
Name: count, dtype: int64


In [ ]:
#Oversample my dataset to deal with imbalanced labels
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(df[['data']], df['labels'])


In [ ]:
y_resampled.value_counts()

labels
entertainment    511
business         511
sport            511
politics         511
tech             511
Name: count, dtype: int64

In [ ]:
data= X_resampled
data['labels']=y_resampled
data.head()

,data,labels
0,Musicians to tackle US red tape Musicians gro...,entertainment
1,"U2s desire to be number one U2, who have won ...",entertainment
2,Rocker Doherty in on-stage fight Rock singer ...,entertainment
3,Snicket tops US box office chart The film ada...,entertainment
4,"Oceans Twelve raids box office Oceans Twelve,...",entertainment


In [ ]:
data.labels.value_counts()

labels
entertainment    511
business         511
sport            511
politics         511
tech             511
Name: count, dtype: int64

In [ ]:
#data.labels to num using label encoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['Labels'] = le.fit_transform(data['labels'])
le.classes_

array(['business', 'entertainment', 'politics', 'sport', 'tech'],
      dtype=object)

In [ ]:
data.Labels.value_counts()

Labels
1    511
0    511
3    511
2    511
4    511
Name: count, dtype: int64

In [ ]:
Data=data['data']
Data.head()

0    Musicians to tackle US red tape  Musicians gro...
1    U2s desire to be number one  U2, who have won ...
2    Rocker Doherty in on-stage fight  Rock singer ...
3    Snicket tops US box office chart  The film ada...
4    Oceans Twelve raids box office  Oceans Twelve,...
Name: data, dtype: object

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Data, data['Labels'], test_size=0.2, random_state=42)

In [ ]:
import torch
y_train = torch.tensor(y_train.values)
y_test = torch.tensor(y_test.values)

In [ ]:
# Tokenize data
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
X_train_encodings = tokenizer(X_train.to_list(), truncation=True, padding=True, return_tensors='pt')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
X_test_encodings = tokenizer(X_test.to_list(), truncation=True, padding=True, return_tensors='pt')

In [ ]:
from torch.utils.data import Dataset, DataLoader

class DocumentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item


In [ ]:
# Create dataset and dataloader
Train_dataset = DocumentDataset(X_train_encodings, y_train)
Test_dataset = DocumentDataset(X_test_encodings, y_test)
train_dataloader = DataLoader(Train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(Test_dataset, batch_size=16, shuffle=False)

In [ ]:
len(le.classes_)

5

In [ ]:
from transformers import get_scheduler
from transformers import BertForSequenceClassification, AdamW
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(le.classes_), torch_dtype=torch.float32)
# Define the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Define the scheduler
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)


In [ ]:
# Move model to GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        # Debugging: Print the batch structure
        print(batch)

        # Move batch to device
        batch = {k: v.to(device) for k, v in batch.items()}

        # Debugging: Print tensor shapes and values
        for key, value in batch.items():
            print(f"{key}: shape {value.shape}, values {value}")

        # Forward pass
        outputs = model(**batch)
        loss = outputs.loss

        # Backward pass
        loss.backward()

        # Update parameters
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    print(f"Epoch {epoch + 1}/{num_epochs} completed. Loss: {loss.item()}")


<ipython-input-16-bae2d1ddaf83>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-16-bae2d1ddaf83>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Streaming output truncated to the last 5000 lines.
        ...,
        [  101,  4300, 21664,  ...,  2000,  1996,   102],
        [  101,  3424,  1011,  ...,     0,     0,     0],
        [  101, 12196,  9422,  ...,  2056,  2045,   102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([3, 0, 0, 4, 4, 4, 1, 3, 2, 1, 2, 2, 2, 1, 4, 2])}
input_ids: shape torch.Size([16, 512]), values tensor([[  101, 26447,  3138,  ...,     0,     0,     0],
        [  101,  2149,  5814,  ...,     0,     0,     0],
        [  101,  5410,  7922,  ...,     0,     0,     0

In [ ]:
from sklearn.metrics import accuracy_score

model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in test_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)
        predictions.extend(preds.cpu().numpy())
        true_labels.extend(batch['labels'].cpu().numpy())

accuracy = accuracy_score(true_labels, predictions)
print(f"Validation Accuracy: {accuracy}")

<ipython-input-16-bae2d1ddaf83>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-16-bae2d1ddaf83>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Validation Accuracy: 0.9882583170254403


In [ ]:
# Save the tokenizer
tokenizer.save_pretrained('./model')

# Save the model
model.save_pretrained('./model')

In [ ]:
# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('./model')

# Load the model
model = BertForSequenceClassification.from_pretrained('./model')

# Ensure the model is in evaluation mode
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e